# Bibliotecas

In [ ]:
import numpy as np

# Abrindo dados

In [ ]:
x = np.float64(np.load(open('data/cnn/x.npy', 'rb')))
y = np.int8(np.load(open('data/cnn/y.npy', 'rb')))

In [ ]:
from keras.utils import to_categorical

y_one_hot = to_categorical(y, 3)
y_one_hot

# Criar Modelo

In [ ]:
from keras import layers, models

input = layers.Input(shape=(501, 6, 1))

conv1 = layers.Conv2D(100, (5, 5))(input)
conv1 = layers.AveragePooling2D((2, 2))(conv1)
conv1 = layers.Flatten()(conv1)
conv2 = layers.Conv2D(100, (25, 5))(input)
conv2 = layers.AveragePooling2D((2, 2))(conv2)
conv2 = layers.Flatten()(conv2)
conv3 = layers.Conv2D(100, (100, 5))(input)
conv3 = layers.AveragePooling2D((2, 2))(conv3)
conv3 = layers.Flatten()(conv3)

conv_layers = layers.Concatenate()([conv1, conv2, conv3])

modelo = layers.Dropout(0.5)(conv_layers)
modelo = layers.Dense(1000, 'relu')(modelo)
modelo = layers.Dropout(0.5)(modelo)
modelo = layers.Dense(1000, 'relu')(modelo)
modelo = layers.Dense(3, 'softmax')(modelo)

modelo = models.Model(input, modelo)
modelo.compile(loss='categorical_crossentropy', metrics=['accuracy'])
modelo.summary()

# Treinar Modelo

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

tensorboard_callback = TensorBoard(log_dir="./logs/customModel/")
earlystop_cb = EarlyStopping(monitor='val_accuracy', patience=20, min_delta=0.001, verbose=True)
modelcheckpoint_cb = ModelCheckpoint('/mnt/c/Users/aurel/OneDrive/Documentos/Projetos/tradingPredictor/models/CustomModel1.h5', 'val_accuracy', save_best_only=True)

modelo.fit(
    x,
    y_one_hot,
    shuffle=True, 
    epochs=500,
    batch_size=256,
    validation_split=0.2,
    callbacks=[tensorboard_callback, earlystop_cb, modelcheckpoint_cb]
)